In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

# Crie um objeto Service apontando para o executável do chromedriver
servico = Service(executable_path=r'./chromedriver.exe')

# Crie o navegador Chrome usando o objeto Service
navegador = webdriver.Chrome(service=servico)

# Navegue para o site da Shopee
navegador.get('https://shopee.com.br/daitecom?categoryId=100637&entryPoint=ShopByPDP&itemId=20169223357')

In [2]:
time.sleep(3)

links = []  # Lista para armazenar os links
ultima_pagina = None

while True:
    elementos_div = navegador.find_elements(By.CSS_SELECTOR, "div.shop-search-result-view__item.col-xs-2-4")

    for elemento_div in elementos_div:
        elemento_a = elemento_div.find_element(By.CSS_SELECTOR, "a[data-sqe='link']")
        link = elemento_a.get_attribute('href')
        
        # Verifica se o link já foi coletado anteriormente
        if link not in links:
            links.append(link)

    if ultima_pagina is not None and ultima_pagina == links[-1]:
        print(f"Links coletados: {len(links)}")
        break

    ultima_pagina = links[-1]

    botao_passar_pagina = navegador.find_elements(By.CSS_SELECTOR, "button.shopee-icon-button.shopee-icon-button--right")

    navegador.execute_script("arguments[0].click();", botao_passar_pagina[0])
    time.sleep(6)  # Aguardar 6 segundos para o conteúdo carregar na próxima página

print("Coleta de links finalizada.")

Links coletados: 81
Coleta de links finalizada.


In [3]:
lista_de_info = []

def encontrar_elemento(selector, default="Não exibe"):
    try:
        return navegador.find_element(By.CSS_SELECTOR, selector).text
    except NoSuchElementException:
        return default
    
numero_de_repeticao = 0

for link in links:
    numero_de_repeticao += 1
    navegador.get(link)
    time.sleep(7)  # Aguardar 7 segundos antes de localizar os elementos

    produto = encontrar_elemento("[class='pqTWkA']")
    nome = encontrar_elemento("div._44qnta span")
    vendas = encontrar_elemento("[class='e9sAa2']")
    avaliacoes = encontrar_elemento("[class='_1k47d8']", 0)
    estrelas = encontrar_elemento("[class='_1k47d8 _046PXf']", "Não exibe")

    dicionario = {'nome_do_produto': nome, 'preço': produto, 'vendas': vendas, 'avaliacoes': avaliacoes, 'estrelas': estrelas, 'link': link}
    print(numero_de_repeticao,dicionario)
    lista_de_info.append(dicionario)

navegador.quit()
print(lista_de_info)

1 {'nome_do_produto': 'Airpods 3 Gen Fones De Ouvido GPS Bluetooth Sem Fio Renomear Super Bass Com Microfone', 'preço': 'R$79,99', 'vendas': '324', 'avaliacoes': '96', 'estrelas': '4.6', 'link': 'https://shopee.com.br/Airpods-3-Gen-Fones-De-Ouvido-GPS-Bluetooth-Sem-Fio-Renomear-Super-Bass-Com-Microfone-i.240543476.20642347226?sp_atk=bf92d25b-0f70-4cc9-a403-b66165b2c53f&xptdk=bf92d25b-0f70-4cc9-a403-b66165b2c53f'}
2 {'nome_do_produto': '20W PD Carregador USB-C para iPhone', 'preço': 'R$15,80 - R$47,80', 'vendas': '1,5mil', 'avaliacoes': '536', 'estrelas': '4.8', 'link': 'https://shopee.com.br/20W-PD-Carregador-USB-C-para-iPhone-i.240543476.12068744027?sp_atk=f41c9a56-a0e4-478c-a861-8758a207478a&xptdk=f41c9a56-a0e4-478c-a861-8758a207478a'}
3 {'nome_do_produto': 'AirPods Pro 2 TWS Fone De Ouvido Estéreo Sem Fio Pro2 GPS Bluetooth Renomear Baixo Com Microfone', 'preço': 'R$95,00', 'vendas': '512', 'avaliacoes': '185', 'estrelas': '4.8', 'link': 'https://shopee.com.br/AirPods-Pro-2-TWS-Fone

21 {'nome_do_produto': 'Anycast G2X Chromecast WIFI Wireless cast HDMI para TV', 'preço': 'R$58,40', 'vendas': '16,1mil', 'avaliacoes': '5,3mil', 'estrelas': '4.2', 'link': 'https://shopee.com.br/Anycast-G2X-Chromecast-WIFI-Wireless-cast-HDMI-para-TV-i.240543476.8531626114?sp_atk=177f8490-35c4-498a-ba2d-548c615e9913&xptdk=177f8490-35c4-498a-ba2d-548c615e9913'}
22 {'nome_do_produto': 'IWO T500 Smartwatch Homens Mulheres Top Relógio Inteligente Monitor De Frequência Cardíaca Whatsapp Mensagem Lembrete Atividade Esportiva Tacker Para IOS Android VS T5', 'preço': 'R$59,90', 'vendas': '187', 'avaliacoes': '74', 'estrelas': '4.5', 'link': 'https://shopee.com.br/IWO-T500-Smartwatch-Homens-Mulheres-Top-Rel%C3%B3gio-Inteligente-Monitor-De-Frequ%C3%AAncia-Card%C3%ADaca-Whatsapp-Mensagem-Lembrete-Atividade-Esportiva-Tacker-Para-IOS-Android-VS-T5-i.240543476.9129031764?sp_atk=63670e87-05e3-437a-94d3-7b9c7bdd0949&xptdk=63670e87-05e3-437a-94d3-7b9c7bdd0949'}
23 {'nome_do_produto': 'Fingertip Pulse O

39 {'nome_do_produto': 'Dongle Anycast com Wi-Fi / Miracast HDMI 1080P / Airplay', 'preço': 'R$49,50', 'vendas': '2,2mil', 'avaliacoes': '886', 'estrelas': '4.3', 'link': 'https://shopee.com.br/Dongle-Anycast-com-Wi-Fi-Miracast-HDMI-1080P-Airplay-i.240543476.6628897232?sp_atk=d14ec4bf-a4a5-4a8c-a3e9-a572b578684e&xptdk=d14ec4bf-a4a5-4a8c-a3e9-a572b578684e'}
40 {'nome_do_produto': "Y80 Fone De Ouvido Bluetooth Sem Fio IPX4 Fones Intra-Auriculares À Prova D'água Sport HiFi Stereo Earset Para Xiaomi", 'preço': 'R$39,99', 'vendas': '3', 'avaliacoes': '1', 'estrelas': '5.0', 'link': "https://shopee.com.br/Y80-Fone-De-Ouvido-Bluetooth-Sem-Fio-IPX4-Fones-Intra-Auriculares-%C3%80-Prova-D'%C3%A1gua-Sport-HiFi-Stereo-Earset-Para-Xiaomi-i.240543476.16686269763?sp_atk=601ce1fc-0370-4184-aa12-26b8c6e7e1d5&xptdk=601ce1fc-0370-4184-aa12-26b8c6e7e1d5"}
41 {'nome_do_produto': 'Airplus Pro6 TWS Bluetooth Fone de ouvido sem fio Fone de ouvido para IOS Android', 'preço': 'R$39,90', 'vendas': '230', 'avalia

59 {'nome_do_produto': 'Correia de nylon elástica para Apple Watch Series 6 5 Se 4 3 Pulseira ajustável para IWatch 38mm 42mm 44mm 40mm Pulseira esportiva bandas', 'preço': 'R$15,99', 'vendas': '222', 'avaliacoes': '88', 'estrelas': '4.8', 'link': 'https://shopee.com.br/Correia-de-nylon-el%C3%A1stica-para-Apple-Watch-Series-6-5-Se-4-3-Pulseira-ajust%C3%A1vel-para-IWatch-38mm-42mm-44mm-40mm-Pulseira-esportiva-bandas-i.240543476.8465822535?sp_atk=81a68336-47be-43b8-a172-aba690f92d3c&xptdk=81a68336-47be-43b8-a172-aba690f92d3c'}
60 {'nome_do_produto': '10 inch sem fio Bluetooth Conjunto de teclado e mouse de portátil leve para telefone iPad colorido', 'preço': 'R$77,00 - R$98,00', 'vendas': '9', 'avaliacoes': '5', 'estrelas': '4.8', 'link': 'https://shopee.com.br/10-inch-sem-fio-Bluetooth-Conjunto-de-teclado-e-mouse-de-port%C3%A1til-leve-para-telefone-iPad-colorido-i.240543476.11621104449?sp_atk=d51dc469-1ddc-4d5d-8d6d-12d0b0c1e20c&xptdk=d51dc469-1ddc-4d5d-8d6d-12d0b0c1e20c'}
61 {'nome_do_

78 {'nome_do_produto': "Capa Protetora De Sílica À Prova D 'Água Para Fones De Ouvido Airpods Apple Airpods 12", 'preço': 'R$11,99', 'vendas': '1,4mil', 'avaliacoes': '538', 'estrelas': '4.8', 'link': "https://shopee.com.br/Capa-Protetora-De-S%C3%ADlica-%C3%80-Prova-D-'%C3%81gua-Para-Fones-De-Ouvido-Airpods-Apple-Airpods-12-i.240543476.4962336257?sp_atk=101473dc-6edb-4a05-85db-a2094bacbc75&xptdk=101473dc-6edb-4a05-85db-a2094bacbc75"}
79 {'nome_do_produto': 'Fone De Ouvido Airdots Com Bluetooth Sem Fio Tws Com Caixa Portátil Stereo Bass', 'preço': 'R$39,00', 'vendas': '272', 'avaliacoes': '110', 'estrelas': '4.8', 'link': 'https://shopee.com.br/Fone-De-Ouvido-Airdots-Com-Bluetooth-Sem-Fio-Tws-Com-Caixa-Port%C3%A1til-Stereo-Bass-i.240543476.6228896845?sp_atk=0737c4c8-23d7-4c05-be48-362c0c5478dc&xptdk=0737c4c8-23d7-4c05-be48-362c0c5478dc'}
80 {'nome_do_produto': 'G12 TV Stick 4K Tela espelho 2.4G WiFi Display Dongle Anycast Miracast para 1080P Decodificação de áudio Google HD', 'preço': '

In [4]:
for dicionario in lista_de_info:
    df = pd.DataFrame.from_dict(dicionario, orient='index', columns=['valor'])
    display(df)

,valor
nome_do_produto,Airpods 3 Gen Fones De Ouvido GPS Bluetooth Se...
preço,"R$79,99"
vendas,324
avaliacoes,96
estrelas,4.6
link,https://shopee.com.br/Airpods-3-Gen-Fones-De-O...


,valor
nome_do_produto,20W PD Carregador USB-C para iPhone
preço,"R$15,80 - R$47,80"
vendas,"1,5mil"
avaliacoes,536
estrelas,4.8
link,https://shopee.com.br/20W-PD-Carregador-USB-C-...


,valor
nome_do_produto,AirPods Pro 2 TWS Fone De Ouvido Estéreo Sem F...
preço,"R$95,00"
vendas,512
avaliacoes,185
estrelas,4.8
link,https://shopee.com.br/AirPods-Pro-2-TWS-Fone-D...


,valor
nome_do_produto,4 Em 1 OTG USB3.0 flash drive 64gb 128gb 256gb...
preço,"R$79,00 - R$149,00"
vendas,0
avaliacoes,0
estrelas,Não exibe
link,https://shopee.com.br/4-Em-1-OTG-USB3.0-flash-...


,valor
nome_do_produto,[Display português] Smartwatch Q19 Dual Camera...
preço,"R$87,00"
vendas,19
avaliacoes,6
estrelas,4.5
link,https://shopee.com.br/-Display-portugu%C3%AAs-...


...

,valor
nome_do_produto,Lenovo LP5 TWS Fone De Ouvido Bluetooth Superb...
preço,"R$99,00"
vendas,1
avaliacoes,1
estrelas,5.0
link,https://shopee.com.br/Lenovo-LP5-TWS-Fone-De-O...


,valor
nome_do_produto,Capa Protetora De Sílica À Prova D 'Água Para ...
preço,"R$11,99"
vendas,"1,4mil"
avaliacoes,538
estrelas,4.8
link,https://shopee.com.br/Capa-Protetora-De-S%C3%A...


,valor
nome_do_produto,Fone De Ouvido Airdots Com Bluetooth Sem Fio T...
preço,"R$39,00"
vendas,272
avaliacoes,110
estrelas,4.8
link,https://shopee.com.br/Fone-De-Ouvido-Airdots-C...


,valor
nome_do_produto,G12 TV Stick 4K Tela espelho 2.4G WiFi Display...
preço,"R$89,99"
vendas,321
avaliacoes,108
estrelas,4.3
link,https://shopee.com.br/G12-TV-Stick-4K-Tela-esp...


,valor
nome_do_produto,Y30 TWS Fones De Ouvido Sem Fio Com Bluetooth ...
preço,"R$27,50"
vendas,"7,8mil"
avaliacoes,"2,1mil"
estrelas,4.8
link,https://shopee.com.br/Y30-TWS-Fones-De-Ouvido-...


In [5]:
compilada = None  # Inicializa compilada como None

for dicionario in lista_de_info:
    df = pd.DataFrame.from_dict(dicionario, orient='index', columns=['valor'])
    df_invertido = df.transpose()  # ou df.T
    if compilada is None:
        compilada = df_invertido  # Se compilada é None, atribui o primeiro DataFrame diretamente
    else:
        lista_tabelas = [compilada, df_invertido]
        compilada = pd.concat(lista_tabelas)  # Concatena os DataFrames
display(compilada)

,nome_do_produto,preço,vendas,avaliacoes,estrelas,link
valor,Airpods 3 Gen Fones De Ouvido GPS Bluetooth Se...,"R$79,99",324,96,4.6,https://shopee.com.br/Airpods-3-Gen-Fones-De-O...
valor,20W PD Carregador USB-C para iPhone,"R$15,80 - R$47,80","1,5mil",536,4.8,https://shopee.com.br/20W-PD-Carregador-USB-C-...
valor,AirPods Pro 2 TWS Fone De Ouvido Estéreo Sem F...,"R$95,00",512,185,4.8,https://shopee.com.br/AirPods-Pro-2-TWS-Fone-D...
valor,4 Em 1 OTG USB3.0 flash drive 64gb 128gb 256gb...,"R$79,00 - R$149,00",0,0,Não exibe,https://shopee.com.br/4-Em-1-OTG-USB3.0-flash-...
valor,[Display português] Smartwatch Q19 Dual Camera...,"R$87,00",19,6,4.5,https://shopee.com.br/-Display-portugu%C3%AAs-...
...,...,...,...,...,...,...
valor,Lenovo LP5 TWS Fone De Ouvido Bluetooth Superb...,"R$99,00",1,1,5.0,https://shopee.com.br/Lenovo-LP5-TWS-Fone-De-O...
valor,Capa Protetora De Sílica À Prova D 'Água Para ...,"R$11,99","1,4mil",538,4.8,https://shopee.com.br/Capa-Protetora-De-S%C3%A...
valor,Fone De Ouvido Airdots Com Bluetooth Sem Fio T...,"R$39,00",272,110,4.8,https://shopee.com.br/Fone-De-Ouvido-Airdots-C...
valor,G12 TV Stick 4K Tela espelho 2.4G WiFi Display...,"R$89,99",321,108,4.3,https://shopee.com.br/G12-TV-Stick-4K-Tela-esp...


In [6]:
compilada.to_excel('Importsprodutos.xlsx',index = False)